**Data preprocessing, Create Label and Features**

In [1]:
#import dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
DATA_PATH = os.path.join("MP_DATA") # paht of the exported data
with open("word_list.txt") as file:
  action = file.readlines()
for i,word in enumerate(action):
  action[i] = word.replace("\n","")
actions = np.array(action)

no_sequences = 30 # take 30 video to collect data for each action
sequence_length = 30

In [3]:
sequences,lables = [], []
# actions = np.array(os.listdir(os.path.join(DATA_PATH)))
actions = np.array(["go","she","like","me"])
label_map = {lable:num for num,lable in enumerate(actions)}
for action in actions:
  action_file = os.listdir(os.path.join(DATA_PATH,action))
  for sequence in range(len(action_file)):#should change to the number of file
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),f"{frame_num}.npy"))
      window.append(res)
    sequences.append(window)
    lables.append(label_map[action])
    
X = np.array(sequences)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)
  

In [4]:
print(X_train.shape)
print(y_train.shape)

(114, 30, 1662)
(114, 4)


**Train the AI**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflowjs as tfjs

In [6]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [10]:
model = Sequential() #
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) #frame and all data shape
# model.add(LSTM(128,return_sequences=True,activation='relu')) 
model.add(LSTM(64,return_sequences=False,activation='relu')) # return false because next layer is dense
model.add(Dropout(0.3)) 
# model.add(Dense(64,activation='relu')) 
model.add(Dense(64,activation='relu')) 
model.add(Dense(actions.shape[0],activation='softmax')) # return output by how many action do we have => [0.7,0.1,0.2] / likely be 'hello'

#loss = categorical => multiple class 
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#epochs increase relatively the number of data 
model.fit(X_train,y_train,epochs=150,callbacks=[tb_callback])

Epoch 1/150
4/4 [==============================] - 7s 1s/step - loss: 3.0201 - categorical_accuracy: 0.2456
Epoch 2/150
4/4 [==============================] - 1s 287ms/step - loss: 1.5107 - categorical_accuracy: 0.2193
Epoch 3/150
4/4 [==============================] - 1s 216ms/step - loss: 1.3628 - categorical_accuracy: 0.3333
Epoch 4/150
4/4 [==============================] - 1s 225ms/step - loss: 1.2891 - categorical_accuracy: 0.3947
Epoch 5/150
4/4 [==============================] - 1s 230ms/step - loss: 1.2464 - categorical_accuracy: 0.3947
Epoch 6/150
4/4 [==============================] - 1s 217ms/step - loss: 1.0716 - categorical_accuracy: 0.5175
Epoch 7/150
4/4 [==============================] - 1s 179ms/step - loss: 1.0412 - categorical_accuracy: 0.4737
Epoch 8/150
4/4 [==============================] - 1s 209ms/step - loss: 0.7855 - categorical_accuracy: 0.6667
Epoch 9/150
4/4 [==============================] - 1s 215ms/step - loss: 0.6499 - categorical_accuracy: 0.6842
Epoc

In [ ]:
#watch the log
%cd logs/train
%tensorboard --logdir=.

In [11]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 260       
Total params: 479,556
Trainable params: 479,556
Non-trainable params: 0
_________________________________________________________________


**save module**

In [12]:
model.save("../../action.h5")
tfjs.converters.save_keras_model(model, "../../tfjs")

In [ ]:
# reload model weight
model.load_weights('action.h5')

In [ ]:
#load model
from tensorflow import keras
model = keras.models.load_model('path/to/location')

**eval**

In [13]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

1.0

In [17]:
mp_holistic = mp.solutions.holistic # holistic module
mp_drawing = mp.solutions.drawing_utils # drawing utilities
def mediapipe_detection(image,model):
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # color conversion
  image.flags.writeable = False
  results = model.process(image) # process image
  image.flags.writeable = True
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # color conversion
  return image,results
def draw_landmarks(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
def draw_styles_landmarked(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                            mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,10),thickness=1,circle_radius=1))
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
def extract_keypoint(result):
  pose = np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
  face = np.array([[res.x,res.y,res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
  left_hand = np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
  right_hand = np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
  return np.concatenate([pose,face,left_hand,right_hand])
colors = [(245,117,16), (117,245,16), (16,117,245),(44,217,235)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [20]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoint(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            sequence = []
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [16]:
cap.release()
cv2.destroyAllWindows()